In [3]:
!python -m pip install git+https://github.com/ChrisWaites/pyvacy

  Cloning https://github.com/ChrisWaites/pyvacy to /tmp/pip-req-build-ds0n7zat
  Running command git clone -q https://github.com/ChrisWaites/pyvacy /tmp/pip-req-build-ds0n7zat
  Created wheel for pyvacy: filename=pyvacy-0.0.1-py3-none-any.whl size=13541 sha256=15a58a48bd21f4bdce0ed06850f4ef6ca2733ae69f17cf464c34b6805aa1dd74
  Stored in directory: /tmp/pip-ephem-wheel-cache-35gtvi1e/wheels/40/eb/29/76f7941eb0c1fd78c6c738d1c7bde62b3cd8cbf364ad1686a3
Successfully built pyvacy


In [4]:
from torchvision import datasets 
from tqdm.notebook import tqdm
from torchvision import datasets, transforms, models 
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data import random_split
from torch.utils.data.dataloader import DataLoader

In [5]:


import argparse
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset
from torchvision import datasets, transforms

from pyvacy import optim, analysis


# Deterministic output
torch.manual_seed(0)
np.random.seed(0)

class Flatten(nn.Module):
    def forward(self, inp):
        return inp.reshape(inp.shape[0], -1)

class Classifier(nn.Module):
    def __init__(self, input_dim,device='cpu'):
        super(Classifier, self).__init__()
        self.conv1 =torch.nn.Conv2d(3, 32, 3, 1)
        self.conv2 = torch.nn.Conv2d(32, 64, 3, 1) 
        self.dropout = torch.nn.Dropout2d(0.25)
        self.fc1 = torch.nn.Linear(147456, 128)
        self.fc2 = torch.nn.Linear(128, 4)

    def forward(self, x):
        x = torch.nn.functional.relu(self.conv1(x))
        x = torch.nn.functional.relu(self.conv2(x))
        x = torch.nn.functional.max_pool2d(x, 2)
        x = self.dropout(x)
        x = torch.flatten(x, 1)
        x = torch.nn.functional.relu(self.fc1(x))
        x = self.fc2(self.dropout(x))
        output = torch.nn.functional.log_softmax(x, dim=1)
        return output

   


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
from pyvacy import optim, analysis, sampling

In [8]:
from zipfile import ZipFile

with ZipFile('/content/drive/MyDrive/archive.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()

In [9]:
import os
train_dir = '/content/train'
val_dir = '/content/val'
test_dir = '/content/val'
classes = os.listdir(train_dir)
print(classes)
print(len(classes))

['TURBERCULOSIS', 'COVID19', 'NORMAL', 'PNEUMONIA']
4


In [23]:
train_transform=transforms.Compose([
        transforms.RandomRotation(10),      # rotate +/- 10 degrees
        transforms.RandomHorizontalFlip(),  # reverse 50% of images
        transforms.Resize(100),             # resize shortest side
        transforms.CenterCrop(100),         # crop longest side
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],
                             [0.229, 0.224, 0.225])
])


def train(params):
  
  trainset = ImageFolder(train_dir, transform=train_transform)
  valset = ImageFolder(val_dir, transform=train_transform)
  testset = ImageFolder(test_dir, transform=train_transform)
  train_ds, val_ds, test_ds = trainset, valset, testset
  train_loader = DataLoader(train_ds, batch_size, shuffle=True, num_workers=4, pin_memory=True)
  val_loader = DataLoader(val_ds, batch_size*2, num_workers=4, pin_memory=True)
  test_loader = DataLoader(test_ds, batch_size*2, num_workers=4, pin_memory=True)


  classifier = Classifier(input_dim=np.prod(trainset[0][0].shape),device=params['device'])

  optimizer = optim.DPSGD(
      l2_norm_clip=params['l2_norm_clip'],
      noise_multiplier=params['noise_multiplier'],
      params=classifier.parameters(),
      lr=params['lr'],
      weight_decay=params['l2_penalty'],
      minibatch_size=params['minibatch_size'],
      microbatch_size=params['microbatch_size'],
      
      )
  loss_function = nn.NLLLoss()
  minibatch_loader, microbatch_loader = sampling.get_data_loaders(
    params['minibatch_size'],
    params['microbatch_size'],
    params['iterations']
    )
  iteration = 0
  for X_minibatch, y_minibatch in minibatch_loader(trainset):
        optimizer.zero_grad()
        for X_microbatch, y_microbatch in microbatch_loader(TensorDataset(X_minibatch, y_minibatch)):
            X_microbatch = X_microbatch.to(params['device'])
            y_microbatch = y_microbatch.to(params['device'])

            #optimizer.zero_microbatch_grad()
            loss = loss_function(classifier(X_microbatch), y_microbatch)
            loss.backward()
            #optimizer.microbatch_step()
        optimizer.step()

        if iteration % 10 == 0:
            print('[Iteration %d/%d] [Loss: %f]' % (iteration, params['iterations'], loss.item()))
        iteration += 1
  return classifier

    

In [32]:
import sys
sys.argv=['']
del sys

In [30]:
 testset = ImageFolder(test_dir, transform=train_transform)

In [33]:
if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--delta', type=float, default=1e-5, help='delta for epsilon calculation (default: 1e-5)')
    parser.add_argument('--device', type=str, default=('cuda' if torch.cuda.is_available() else 'cpu'), help='whether or not to use cuda (default: cuda if available)')
    parser.add_argument('--iterations', type=int, default=10, help='number of iterations to train (default: 14000)')
    parser.add_argument('--l2-norm-clip', type=float, default=1., help='upper bound on the l2 norm of gradient updates (default: 0.1)')
    parser.add_argument('--l2-penalty', type=float, default=0.001, help='l2 penalty on model weights (default: 0.001)')
    parser.add_argument('--lr', type=float, default=0.15, help='learning rate (default: 0.15)')
    parser.add_argument('--microbatch-size', type=int, default=1, help='input microbatch size for training (default: 1)')
    parser.add_argument('--minibatch-size', type=int, default=256, help='input minibatch size for training (default: 256)')
    parser.add_argument('--noise-multiplier', type=float, default=1.1, help='ratio between clipping bound and std of noise applied to gradients (default: 1.1)')
    params = vars(parser.parse_args())

    classifier = train(params)

    with open('dp_classifier.dat', 'wb') as f:
        torch.save(classifier, f)


    X, y = next(iter(DataLoader(testset)))
    # X, y  = X.to('cuda'), y.to('cuda')

    y_pred = classifier(X).max(1)[1]

    count = 0.
    correct = 0.
    for pred, actual in zip(y_pred, y):
        if pred.item() == actual.item():
            correct += 1.
        count += 1.
    print('Test Accuracy: {}'.format(correct / count))


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


[Iteration 0/10] [Loss: 1.351786]
Test Accuracy: 0.0
